In [3]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import sys
sys.path.append('/opt/ml/code/pytorch_image_models_master')
from timm.models import create_model

In [13]:
import torch
import timm
m = timm.create_model('ecaresnet269d', features_only=True, pretrained=True)
o = m(torch.randn(2, 3, 256, 256))
for i in o:
    print(i.shape)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/ecaresnet269d_320_ra2-7baa55cb.pth" to /opt/ml/.cache/torch/hub/checkpoints/ecaresnet269d_320_ra2-7baa55cb.pth
torch.Size([2, 64, 128, 128])
torch.Size([2, 256, 64, 64])
torch.Size([2, 512, 32, 32])
torch.Size([2, 1024, 16, 16])
torch.Size([2, 2048, 8, 8])


In [14]:
!python \
    /opt/ml/code/mmdetection_trash/tools/misc/print_config.py \
    /opt/ml/code/my_code/my_custom.py

Config:
dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=[
            dict(
                type='RandomSizedCrop',
                min_max_height=[180, 180],
                height=256,
                width=256,
                p=0.5)
        ],
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675

In [15]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('/opt/ml/code/my_code/my_custom.py')

PREFIX = '/opt/ml/input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/custom'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [16]:
model = build_detector(cfg.model)

In [17]:
print(type(model))

<class 'mmdet.models.detectors.vfnet.VFNet'>


In [18]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.18s)
creating index...
index created!


In [19]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-19 11:39:01,997 - mmdet - INFO - Start running, host: root@bc5257f7549f, work_dir: /opt/ml/code/my_code/work_dirs/custom
2021-05-19 11:39:01,998 - mmdet - INFO - workflow: [('train', 1)], max: 30 epochs
Done (t=0.89s)
creating index...
index created!


-05-19 11:42:33,305 - mmdet - INFO - Epoch [1][600/655]	lr: 5.000e-03, eta: 1:48:33, time: 0.336, data_time: 0.005, memory: 3624, loss_cls: 0.9580, loss_bbox: 0.5822, loss_bbox_rf: 0.7727, loss: 2.3129, grad_norm: 5.5994
2021-05-19 11:42:50,020 - mmdet - INFO - Epoch [1][650/655]	lr: 5.000e-03, eta: 1:48:05, time: 0.334, data_time: 0.005, memory: 3624, loss_cls: 1.0196, loss_bbox: 0.5513, loss_bbox_rf: 0.7258, loss: 2.2967, grad_norm: 5.5030
2021-05-19 11:42:51,804 - mmdet - INFO - Saving checkpoint at 1 epochs
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 11.6 task/s, elapsed: 57s, ETA:     0s2021-05-19 11:43:52,104 - mmdet - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=1.41s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.97s).
Accumulating evaluation results...
DONE (t=1.89s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.007
Average Precision  (AP) @[ IoU=0.50      | area=   al

KeyboardInterrupt: 

In [ ]:
for fold in range(5):
    classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
    # config file 들고오기
    cfg = Config.fromfile('/opt/ml/code/my_code/my_custom.py')

    PREFIX = '/opt/ml/input/data/'

    # dataset 바꾸기
    cfg.data.train.classes = classes
    cfg.data.train.img_prefix = PREFIX
    cfg.data.train.ann_file = PREFIX + f'train_data{fold}.json'
    cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

    cfg.data.val.classes = classes
    cfg.data.val.img_prefix = PREFIX
    cfg.data.val.ann_file = PREFIX + f'valid_data{fold}.json'
    cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

    cfg.data.test.classes = classes
    cfg.data.test.img_prefix = PREFIX
    cfg.data.test.ann_file = PREFIX + 'test.json'
    cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

    cfg.data.samples_per_gpu = 4

    cfg.seed=2020
    cfg.gpu_ids = [0]
    cfg.work_dir = f'./work_dirs/VFNet_{fold}'

    cfg.model.bbox_head.num_classes = 11

    cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

    model = build_detector(cfg.model)
    datasets = [build_dataset(cfg.data.train)]
    
    train_detector(model, datasets[0], cfg, distributed=False, validate=True)

In [5]:
fold = 3
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('/opt/ml/code/my_code/my_custom_resume.py')

PREFIX = '/opt/ml/input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + f'train_data{fold}.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + f'valid_data{fold}.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/VFNet_{fold}_2'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]


train_detector(model, datasets[0], cfg, distributed=False, validate=True)

Error in callback <function _WandbInit._resume_backend at 0x7f2a831824d0> (for pre_run_cell):


Exception: The wandb backend process has shutdown